In [ ]:
from IPython.display import Image

# CREATE READ UPDATE DELETE

MERN -> Mongo, Express, React, Node.js

# 설치 

! pip install fastapi 

In [ ]:
# First APP
# pip install uvicorn

#  FristApi.py 파일 있습니다

In [ ]:
from fastapi import FastAPI
app = FastAPI()
@app.get("/")
def getItems():
    return ['Item 1', 'Item 2', 'Item 3']

# 실행 방법 
- uvicorn main:app --reload

## 확인 방법 
- http://127.0.0.1:8000/

### swagger UI
- http://127.0.0.1:8000/docs#/default/getItems__get

In [ ]:
Image('./image/firstatpi2.png')

In [ ]:
Image('./image/firstapi1.png')

# fake dataset

In [ ]:
fakeDatabase = {
    1: {'task': 'Clean car'},
    2: {'task': 'Clean my room'},
    3: {'task': 'Clean my computer'},
        
}

# GET Path Parameters

In [ ]:
@app.get("/{id}")
def getItem(id:int):
    return fakeDatabase[id]

# Post PUT and Delete Requests

In [ ]:
#Option # 1
@app.post("/")
def addItem(task:str):
    newId = len(fakeDatabase.keys()) + 1
    fakeDatabase[newId] = {"task":task}
    return fakeDatabase

# 만약... task 뿐만 아니라 여러 컬럼이 있으면 ?
addItem(“task”:str, “priority”:”str”, “rank”:int, ..........)
이걸 다해 ? 노노

# 설치 
- pip install pydantic

# Create schemas.py

In [ ]:
from pydantic import BaseModel
class Item(BaseModel):
    task: str

# 수정 방법 1

In [ ]:
from fastapi import FastAPI
import schemas


#Option # 2
@app.post("/")
def addItem(item:schemas.Item):
    newId = len(fakeDatabase.keys()) + 1
    fakeDatabase[newId] = {"task":item.task}
    return fakeDatabase

# 수정 방법 2

- we can then access the request body as a dictionary and extract the value of the task

In [ ]:
from fastapi import FastAPI, Body
#Option #3
@app.post("/")
def addItem(body = Body()):
   newId = len(fakeDatabase.keys()) + 1
   fakeDatabase[newId] = {"task":body['task']}
   return fakeDatabase


# updating Data

In [ ]:
@app.put("/{id}")
def updateItem(id:int, item:schemas.Item):
    fakeDatabase[id]['task'] = item.task 
    return fakeDatabase


# Deleting Data

In [ ]:
@app.delete("/{id}")
def deleteItem(id:int):
    del fakeDatabase[id]
    return fakeDatabase


# Database and Models


# 설치 
!pip install sqlalchemy

In [ ]:
# database.py

# connection configure

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker
#Create sqlite engine instance
engine = create_engine("sqlite:///todo.db")
#Create declaritive base meta instance
Base = declarative_base()
#Create session local class for session maker
SessionLocal = sessionmaker(bind=engine, expire_on_commit=False)

In [ ]:
# models.py

In [ ]:
from sqlalchemy import Column, Integer, String
from database import Base
class Item(Base):
    __tablename__ = 'items'
    id = Column(Integer, primary_key=True)
    task = Column(String(256))

# 연결 방법

In [ ]:

#main.py
from fastapi import FastAPI,Body, Depends
import schemas
import models
from database import Base, engine, SessionLocal
from sqlalchemy.orm import Session
#This will create our database if it doesent already exists
Base.metadata.create_all(engine)
def get_session():
    session = SessionLocal()
    try:
        yield session
    finally:
        session.close()

# Working with our databse

In [ ]:
@app.get("/")
def getItems(session: Session = Depends(get_session)):
    items = session.query(models.Item).all()
    return items

In [ ]:
# add item

@app.post("/")
def addItem(item:schemas.Item, session = Depends(get_session)):
    item = models.Item(task = item.task)
    session.add(item)
    session.commit()
    session.refresh(item)
    return item

In [ ]:
# retrieve single item 
@app.get("/{id}")
def getItem(id:int, session: Session = Depends(get_session)):
    item = session.query(models.Item).get(id)
    return item

In [ ]:
# update 
@app.put("/{id}")
def updateItem(id:int, item:schemas.Item, session = Depends(get_session)):
    itemObject = session.query(models.Item).get(id)
    itemObject.task = item.task
    session.commit()
    return itemObject

In [ ]:
@app.delete("/{id}")
def deleteItem(id:int, session = Depends(get_session)):
    itemObject = session.query(models.Item).get(id)
    session.delete(itemObject)
    session.commit()
    session.close()
    return 'Item was deleted'
